In [2]:
import pandas as pd

df = pd.read_csv('churn_cleaned.csv')
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,AutoPay,monthly_to_total,tenure_to_total
0,1,0,0,1,29.85,29.85,0,0,1.000000,0.033501
1,34,1,1,0,56.95,1889.50,0,0,0.030140,0.017994
2,2,1,0,0,53.85,108.15,1,0,0.497920,0.018493
3,45,0,1,2,42.30,1840.75,0,1,0.022980,0.024447
4,2,1,0,1,70.70,151.65,1,0,0.466205,0.013188
...,...,...,...,...,...,...,...,...,...,...
7038,24,1,1,0,84.80,1990.50,0,0,0.042602,0.012057
7039,72,1,1,3,103.20,7362.90,0,1,0.014016,0.009779
7040,11,0,0,1,29.60,346.45,0,0,0.085438,0.031751
7041,4,1,0,0,74.40,306.60,1,0,0.242661,0.013046


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tenure            7043 non-null   int64  
 1   PhoneService      7043 non-null   int64  
 2   Contract          7043 non-null   int64  
 3   PaymentMethod     7043 non-null   int64  
 4   MonthlyCharges    7043 non-null   float64
 5   TotalCharges      7043 non-null   float64
 6   Churn             7043 non-null   int64  
 7   AutoPay           7043 non-null   int64  
 8   monthly_to_total  7043 non-null   float64
 9   tenure_to_total   7043 non-null   float64
dtypes: float64(4), int64(6)
memory usage: 550.4 KB


In [4]:
#use pycaret to find an ML algorithm that performs best on the data

from pycaret.classification import setup, compare_models, predict_model, save_model, load_model

automl = setup(df, target='Churn')

,Description,Value
0,Session id,7144
1,Target,Churn
2,Target type,Binary
3,Original data shape,"(7043, 10)"
4,Transformed data shape,"(7043, 10)"
5,Transformed train set shape,"(4930, 10)"
6,Transformed test set shape,"(2113, 10)"
7,Numeric features,9
8,Preprocess,True
9,Imputation type,simple


In [5]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7972,0.8367,0.4923,0.6590,0.5631,0.4346,0.4428,0.6490
ridge,Ridge Classifier,0.7939,0.8314,0.4220,0.6820,0.5209,0.3987,0.4177,0.0120
gbc,Gradient Boosting Classifier,0.7937,0.8314,0.4747,0.6544,0.5490,0.4199,0.4296,0.2350
lda,Linear Discriminant Analysis,0.7935,0.8324,0.4755,0.6538,0.5502,0.4205,0.4297,0.0110
lightgbm,Light Gradient Boosting Machine,0.7826,0.8173,0.4985,0.6117,0.5482,0.4072,0.4115,0.1190
ada,Ada Boost Classifier,0.7819,0.8273,0.4686,0.6184,0.5327,0.3941,0.4008,0.0850
rf,Random Forest Classifier,0.7684,0.7947,0.4526,0.5843,0.5089,0.3607,0.3664,0.2140
knn,K Neighbors Classifier,0.7645,0.7328,0.4274,0.5750,0.4898,0.3413,0.3478,0.0340
et,Extra Trees Classifier,0.7560,0.7668,0.4603,0.5503,0.4999,0.3405,0.3437,0.1280
qda,Quadratic Discriminant Analysis,0.7552,0.8243,0.7331,0.5283,0.6137,0.4415,0.4546,0.0120


In [6]:
#Choose a metric you think is best to use for finding the best model
best_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=7144, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

I think that AUC is the best metric to use to find the best model. We see that logistic regression is the best model.

In [7]:
# save the model to disk

save_model(best_model, 'Logistic_Regression')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['tenure', 'PhoneService',
                                              'Contract', 'PaymentMethod',
                                              'MonthlyCharges', 'TotalCharges',
                                              'AutoPay', 'monthly_to_total',
                                              'tenure_to_total'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values...
                                                               fill_value=None,
           

In [8]:
import pickle

with open('Logistic_Regression.pk', 'wb') as f:
    pickle.dump(best_model, f)

with open('Logistic_Regression.pk', 'rb') as f:
    loaded_model = pickle.load(f)

new_data = df.iloc[-2:-1].copy()
new_data.drop('Churn', axis=1, inplace=True)
loaded_model.predict(new_data)

loaded_lda = load_model('Logistic_Regression')

predict_model(loaded_lda, new_data)

Transformation Pipeline and Model Successfully Loaded


,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,AutoPay,monthly_to_total,tenure_to_total,prediction_label,prediction_score
7041,4,1,0,0,74.400002,306.600006,0,0.242661,0.013046,0,0.507


In [9]:
# create a Python script/file/module with a function that takes a pandas dataframe as an input 
# and returns the probability of churn for each row in the dataframe

In [12]:
## This is the code in our python script

from pycaret.classification import predict_model, load_model

def load_data(filepath):
    """
    Loads data into a DataFrame from a string filepath.
    """
    df = pd.read_csv(filepath)
    return df


def make_predictions(df):
    """
    Uses the pycaret best model to make predictions on data in the df dataframe.
    """
    model = load_model('Logistic_Regression')
    predictions = predict_model(model, data=df)

    # Check the column names
    print(predictions.columns)
    
    # Rename 'prediction_label' to 'Churn_prediction'
    if 'prediction_label' in predictions.columns:
        predictions.rename(columns={'prediction_label': 'Churn_prediction'}, inplace=True)
        
        # Replace values in the new column
        predictions['Churn_prediction'].replace({1: 'Churn', 0: 'No Churn'}, inplace=True)
        
        return predictions['Churn_prediction']
    else:
        raise KeyError("The 'prediction_label' column was not found in the predictions DataFrame")


if __name__ == "__main__":
    df = load_data('new_churn_cleaned.csv')
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

Transformation Pipeline and Model Successfully Loaded


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.4000,0.0000,0.0000,0.0000,0.0000,-0.3636,-0.4082


Index(['customerID', 'tenure', 'PhoneService', 'Contract', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'AutoPay', 'monthly_to_total',
       'tenure_to_total', 'Churn', 'prediction_label', 'prediction_score'],
      dtype='object')
predictions:
0       Churn
1    No Churn
2    No Churn
3    No Churn
4    No Churn
Name: Churn_prediction, dtype: object


We have saved our python script as a .py file.
Now we must modify our new data to fit the features used in the model.

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        5 non-null      object 
 1   tenure            5 non-null      int64  
 2   PhoneService      5 non-null      int64  
 3   Contract          5 non-null      int64  
 4   PaymentMethod     5 non-null      int64  
 5   MonthlyCharges    5 non-null      float64
 6   TotalCharges      5 non-null      float64
 7   AutoPay           5 non-null      int64  
 8   Churn             5 non-null      int64  
 9   monthly_to_total  5 non-null      float64
 10  tenure_to_total   5 non-null      float64
dtypes: float64(4), int64(6), object(1)
memory usage: 568.0+ bytes


In [14]:
new_df = pd.read_csv('new_churn_data_unmodified.csv')
new_df

,customerID,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges
0,9305-CKSKC,22,Yes,Month-to-month,Electronic check,97.40,811.70
1,1452-KNGVK,8,No,One year,Mailed check,77.30,1701.95
2,6723-OKKJM,28,Yes,Month-to-month,Credit card (automatic),28.25,250.90
3,7832-POPKP,62,Yes,Month-to-month,Electronic check,101.70,3106.56
4,6348-TACGU,10,No,Two year,Credit card (automatic),51.15,3440.97


In [15]:
# We are transforming categorical data into numeric

new_df_copy = new_df

new_df_copy['PhoneService'] = df['PhoneService'].replace({'No': 0, 'Yes': 1})
new_df_copy['Contract'] = df['Contract'].replace({'Month-to-month': 0, 'One year': 1, 'Two year': 2})
new_df_copy['PaymentMethod'] = df['PaymentMethod'].replace({'Mailed check': 0, 'Electronic check': 1, 'Bank transfer (automatic)': 2, 'Credit card (automatic)':3})
new_df_copy['AutoPay'] = df['PaymentMethod'].replace({'Mailed check': 0, 'Electronic check': 0, 'Bank transfer (automatic)': 1, 'Credit card (automatic)':1})
new_df_copy['Churn'] = df['Churn'].replace({'No': 0, 'Yes': 1})
new_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   customerID      5 non-null      object 
 1   tenure          5 non-null      int64  
 2   PhoneService    5 non-null      int64  
 3   Contract        5 non-null      int64  
 4   PaymentMethod   5 non-null      int64  
 5   MonthlyCharges  5 non-null      float64
 6   TotalCharges    5 non-null      float64
 7   AutoPay         5 non-null      int64  
 8   Churn           5 non-null      int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 488.0+ bytes


In [16]:
# Now we will add the the combined features from our cleaned data

new_df_copy['monthly_to_total'] = new_df_copy['MonthlyCharges'] / new_df_copy['TotalCharges']
new_df_copy['tenure_to_total'] = new_df_copy['tenure'] / new_df_copy['TotalCharges']

new_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        5 non-null      object 
 1   tenure            5 non-null      int64  
 2   PhoneService      5 non-null      int64  
 3   Contract          5 non-null      int64  
 4   PaymentMethod     5 non-null      int64  
 5   MonthlyCharges    5 non-null      float64
 6   TotalCharges      5 non-null      float64
 7   AutoPay           5 non-null      int64  
 8   Churn             5 non-null      int64  
 9   monthly_to_total  5 non-null      float64
 10  tenure_to_total   5 non-null      float64
dtypes: float64(4), int64(6), object(1)
memory usage: 568.0+ bytes


In [17]:
new_df_copy.to_csv('new_churn_cleaned.csv', index=False)

In [19]:
%run predict_churn.py

In [20]:
from pycaret.classification import predict_model, load_model

def load_data(filepath):
    """
    Loads data into a DataFrame from a string filepath.
    """
    df = pd.read_csv(filepath)
    return df


def make_predictions(df):
    """
    Uses the pycaret best model to make predictions on data in the df dataframe.
    """
    model = load_model('Logistic_Regression')
    predictions = predict_model(model, data=df)

    # Check the column names
    print(predictions.columns)
    
    # Rename 'prediction_label' to 'Churn_prediction'
    if 'prediction_label' in predictions.columns:
        predictions.rename(columns={'prediction_label': 'Churn_prediction'}, inplace=True)
        
        # Replace values in the new column
        predictions['Churn_prediction'].replace({1: 'Churn', 0: 'No Churn'}, inplace=True)
        
        return predictions['Churn_prediction']
    else:
        raise KeyError("The 'prediction_label' column was not found in the predictions DataFrame")


if __name__ == "__main__":
    df = load_data('new_churn_cleaned.csv')
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

Transformation Pipeline and Model Successfully Loaded


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.4000,0.0000,0.0000,0.0000,0.0000,-0.3636,-0.4082


Index(['customerID', 'tenure', 'PhoneService', 'Contract', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'AutoPay', 'monthly_to_total',
       'tenure_to_total', 'Churn', 'prediction_label', 'prediction_score'],
      dtype='object')
predictions:
0       Churn
1    No Churn
2    No Churn
3    No Churn
4    No Churn
Name: Churn_prediction, dtype: object


For this project I needed to create a virtual enviornment with a downgraded version of Python. I used paycaret to compare auto ml models and saved the best model to a pickle file so that it could be accessed later. I uploaded the unmodified new churn data and repeated the same preproccessing that was applied to the original churn dataset. This ensured that the features in the new dataframe matched with the model. I saved the python script with the lr model as a .py file. The predict_churn.py seems to be running, however I am not getting any outputs. The lr model was 80% accurate in it's predictions. Our model predicted [1, 0, 0, 0, 0] with the true vales being [1, 0, 0, 1, 0]. So it have one false negative.